In [1]:
import pandas as pd
from datetime import datetime
import wrds

insiders_df = pd.read_pickle("raw_insiders_pre_merge.pkl")
final_prices_df = pd.read_pickle("raw_prices.pkl")
date_cols = ['trandate', 'first_date', 'end_date', 'last_date']

for col in date_cols:
    final_prices_df[col] = pd.to_datetime(final_prices_df[col])
final_prices_df['cusip'] = final_prices_df['cusip'].astype('category')
insiders_df['industry'] = insiders_df['industry'].astype('category')
insiders_df['sector'] = insiders_df['sector'].astype('category')
insiders_df['cusipi'] = insiders_df['cusipi'].astype('category')
insiders_df = insiders_df.merge(final_prices_df,
                                left_on=['cusipi', 'trandate'],
                                right_on=['cusip', 'trandate'],
                                how='left')
insiders_df = insiders_df[insiders_df['n_periods'] != 0]
# del final_prices_df
insiders_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7663765 entries, 0 to 8716407
Data columns (total 28 columns):
 #   Column      Dtype         
---  ------      -----         
 0   dcn         string        
 1   seqnum      int16         
 2   personid    int32         
 3   owner       category      
 4   rolecode1   category      
 5   rolecode2   category      
 6   rolecode3   category      
 7   rolecode4   category      
 8   cname       category      
 9   ticker      category      
 10  sector      category      
 11  industry    category      
 12  trandate    datetime64[ns]
 13  tprice      float64       
 14  ownership   category      
 15  cleanse     category      
 16  acqdisp     category      
 17  cusipi      category      
 18  cusip       category      
 19  end_date    datetime64[ns]
 20  first_date  datetime64[ns]
 21  last_date   datetime64[ns]
 22  n_periods   int64         
 23  n_miss      int64         
 24  min_ret     float64       
 25  max_ret     float64    

In [2]:
insiders_df[insiders_df['n_miss']!=0]

,dcn,seqnum,personid,owner,rolecode1,rolecode2,rolecode3,rolecode4,cname,ticker,...,cusip,end_date,first_date,last_date,n_periods,n_miss,min_ret,max_ret,cum_return,permno
40,00500461,1,14015998,COTTRELL DAVID M,O,VP,NaN,NaN,LIBERTY LIVEWIRE CORP,NaN,...,04363410,2000-06-30,2000-06-14,2000-06-30,12,1,-0.085714,0.179941,1.798714e-01,88354.0
805,00503482,5,12097706,CLAYTON STEPHEN T,CFO,O,NaN,NaN,TREMONT ADVISERS INC,NaN,...,89472920,2000-06-15,2000-02-02,2000-06-15,93,1,-0.214724,0.209677,2.141588e-02,87635.0
1133,00504113,1,15020725,RUTHERFORD JERRY,DO,NaN,NaN,NaN,TRAVELNOW COM INC,NaN,...,89490A10,2000-06-01,2000-05-25,2000-06-01,4,1,-0.015873,0.016129,-6.816096e-03,88268.0
1134,00504113,2,15020725,RUTHERFORD JERRY,DO,NaN,NaN,NaN,TRAVELNOW COM INC,NaN,...,89490A10,2000-06-02,2000-05-25,2000-06-02,5,1,-0.015873,0.016129,-1.369366e-02,88268.0
1135,00504113,4,15020725,RUTHERFORD JERRY,DO,NaN,NaN,NaN,TRAVELNOW COM INC,NaN,...,89490A10,2000-06-09,2000-05-25,2000-06-09,10,1,-0.022556,0.072581,1.479702e-07,88268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8705548,FIS0112874,769,12706138,ENSAK DAVID A,D,NaN,NaN,NaN,RAPTOR SYSTEMS INC,NaN,...,75381710,1996-08-07,1996-02-07,1996-08-07,126,1,-0.230392,0.242424,-1.059342e-01,83160.0
8705660,FIS0113059,993,12039224,IRESTONE PATRICK,P,CO,D,NaN,SPANLINK COMMUNICATIONS INC,NaN,...,84649210,1996-10-29,1996-04-29,1996-10-29,128,1,-0.148936,0.185185,-2.042577e-01,83419.0
8705661,FIS0113059,994,12039224,IRESTONE PATRICK,P,CO,D,NaN,SPANLINK COMMUNICATIONS INC,NaN,...,84649210,1996-10-29,1996-04-29,1996-10-29,128,1,-0.148936,0.185185,-2.042577e-01,83419.0
8706643,se00107129,1,16015966,GILLESPIE RICHARD J,D,NaN,NaN,NaN,BANK OF PRINCETON,BPRN,...,74179A10,2017-09-20,2017-08-01,2017-09-20,35,1,-0.042465,0.038585,-3.774622e-02,16853.0


In [3]:
insiders_df['seqnum'] = insiders_df['seqnum'].astype('category')
insiders_df['dcn'] = insiders_df['dcn'].astype('category')

In [4]:
etf_prices = pd.read_pickle("etf.pkl")
etf_prices['trandate'] = pd.to_datetime(etf_prices['trandate'])
db = wrds.Connection(wrds_username='asherbaraban')
sector_etfs = {
    '01': "XLF",  # Finance
    '02': "XLV",  # Healthcare
    '03': "IYK",  # Consumer Non-Durable
    '04': "XLY",  # Consumer Services
    '05': "XHB",  # Consumer Durables
    '06': "XLE",  # Energy
    '07': "IYT",  # Transportation
    '08': "XLK",  # Technology
    '09': "XLB",  # Basic Industries
    '10': "XLI",  # Capital Goods
    '11': "XLU",  # Public Utilities
    '99' : "VTI",  # Miscellaneous
    '00': "VTI",  # Not Classified
}
benchmark_tickers = sector_etfs.values()
tickers_placeholder = ', '.join(f"'{ticker}'" for ticker in benchmark_tickers)
query = f"""
        SELECT 
            LEFT(cusip, 8) AS cusip,
            datadate,
            ajexdi,
            trfd,
            prccd,
            tic
        FROM 
            comp_na_daily_all.secd
        WHERE 
            tic IN ({tickers_placeholder}) AND
            datadate = '2023-10-02'
    """
prices = db.raw_sql(query)

ticker_to_cusip = prices.set_index('tic')['cusip'].to_dict()
insiders_df['sector'] = insiders_df['sector'].fillna('99')
insiders_df['sector_ticker'] = insiders_df['sector'].map(sector_etfs)
insiders_df['sector_ticker'].fillna("VTI", inplace=True)
insiders_df['sector_ticker'] = insiders_df['sector_ticker'].astype('category')
insiders_df['sector_cusip'] = insiders_df['sector_ticker'].map(ticker_to_cusip).astype('category')

Loading library list...
Done


In [5]:
insiders_df = insiders_df.merge(etf_prices,
                                left_on=['trandate', 'sector_cusip'],
                                right_on=['trandate','cusip'],
                                how='left',
                                suffixes=('', '_etf'))

In [6]:
insiders_df['net_ret'] = insiders_df['cum_return'] - insiders_df['cum_return_etf']
insiders_df.dropna(subset=['net_ret'], inplace=True)

In [7]:
insiders_df.to_pickle("CRSP_merged.pkl")

In [21]:
# Filter out rows where cum_return is missing
filtered_etf_prices = etf_prices.dropna(subset=['cum_return'])
filtered_etf_prices = filtered_etf_prices[filtered_etf_prices['n_periods'] > 120]
# Sort by trandate
sorted_etf_prices = filtered_etf_prices.sort_values(by='trandate')

# Group by cusip and get the first row of each group
earliest_rows = sorted_etf_prices.groupby('cusip', as_index=False).first()

In [22]:
earliest_rows

,cusip,trandate,end_date,first_date,last_date,n_periods,n_miss,min_ret,max_ret,cum_return
0,46090E10,1999-03-01,1999-09-01,1999-03-10,1999-09-01,122,1,-0.046903,0.051364,0.073472
1,46428719,2003-10-06,2004-04-06,2003-10-10,2004-04-06,122,1,-0.029334,0.025657,0.026052
2,46428720,2000-05-09,2000-11-09,2000-05-19,2000-11-09,121,1,-0.024452,0.032872,-0.001190
3,46428781,2000-06-07,2000-12-07,2000-06-16,2000-12-07,121,1,-0.027273,0.031346,0.012887
4,46428783,2000-06-07,2000-12-07,2000-06-16,2000-12-07,121,1,-0.051923,0.100908,-0.005189
5,78464A88,2006-01-31,2006-07-31,2006-02-06,2006-07-31,121,1,-0.043467,0.044218,-0.148221
6,81369Y20,1998-12-17,1999-06-17,1998-12-22,1999-06-17,121,1,-0.034910,0.028538,0.068245
7,81369Y40,1998-12-17,1999-06-17,1998-12-22,1999-06-17,121,1,-0.030999,0.032583,0.046790
8,81369Y50,1998-12-17,1999-06-17,1998-12-22,1999-06-17,121,1,-0.029934,0.057221,0.101623
9,81369Y60,1998-12-17,1999-06-17,1998-12-22,1999-06-17,121,1,-0.034230,0.040390,0.043714


In [24]:
ticker_to_cusip

{'XLB': '81369Y10',
 'XLU': '81369Y88',
 'XLK': '81369Y80',
 'XLI': '81369Y70',
 'XLF': '81369Y60',
 'XLE': '81369Y50',
 'XLY': '81369Y40',
 'XLV': '81369Y20',
 'IYK': '46428781',
 'VTI': '92290876',
 'IYT': '46428719',
 'XHB': '78464A88'}